In [ ]:
# triage_agent.py
#
# 1. Classifies a support ticket (simple vs complex)
# 2. Generates an automated reply for simple tickets
# 3. Escalates complex tickets by e-mailing the help-desk queue
#
# Prereqs
#   pip install crewai openai python-dotenv
#   export OPENAI_API_KEY="sk-..."
#   python triage_agent.py

import os
from crewai import Agent, Task, Crew
from dotenv import load_dotenv

load_dotenv()

# ────────────────────────────────────────────────────────────────────────────────
# Build the agents
# ────────────────────────────────────────────────────────────────────────────────
classifier = Agent(
    role="TicketClassifier",
    goal="Decide whether a support ticket is SIMPLE or COMPLEX",
    backstory="""You are an expert in customer-support triage.
                 A SIMPLE ticket can be answered with a template;
                 a COMPLEX one needs a human.""",
    allow_delegation=False,
)

responder = Agent(
    role="ResponderBot",
    goal="Draft a friendly, policy-compliant reply for SIMPLE tickets",
    backstory="You write concise, empathetic e-mails in British English.",
)

# ────────────────────────────────────────────────────────────────────────────────
# Define the tasks
# ────────────────────────────────────────────────────────────────────────────────
# The escalation process - i.e. email a human, but could trigger additional agents
def escalate(subject: str, body: str, recipient="helpdesk@company.com"):
    # Placeholder for the escalation process
    print(f"DEMO - i.e. Email {recipient} with [{subject}]")


def triage_ticket(ticket: str, customer_email: str) -> None:
    # Step 1 – classification
    classification_task = Task(
        description=f"""Classify this ticket as SIMPLE or COMPLEX and
        respond only with the single word SIMPLE or COMPLEX:\n\n{ticket}""",
        expected_output="SIMPLE or COMPLEX",
        agent=classifier,
    )
    result = Crew(tasks=[classification_task]).kickoff()
    label = result.raw.strip().upper()

    if label == "SIMPLE":
        # Step 2 – auto-reply
        # In real deployments, these answers would be personalised and policy-aware, drawing from internal documents,
        # company information or systems. We’ll cover how to do that in [a later chapter].
        # For now, we’re keeping it generic to focus on how agents think and act."
        reply_task = Task(
            description=f"""Write a polite reply to this customer
            (less than 150 words) resolving their issue, sign off as 'Support Team'.\n\n{ticket}""",
            expected_output="A polite email in British English",
            agent=responder,
        )
        draft = Crew(tasks=[reply_task]).kickoff().raw
        print("\n--- AUTOMATED REPLY ---\n", draft, "\n")
        # In production you’d send the e-mail here.
    else:
        # Step 3 – escalate
        escalate("Complex ticket needs human review", ticket)
        print("Ticket escalated to help-desk queue.")


# ────────────────────────────────────────────────────────────────────────────────
# Demo run
# ────────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    sample_ticket = """
    Hi, my parcel arrived damaged and the replacement you promised
    last month never showed up. I need it before Friday or I'll cancel.
    """
    triage_ticket(sample_ticket, "alice@example.com")


In [ ]:
sample_ticket = """
Hi, my parcel arrived damaged and the replacement you promised
last month never showed up. I need it before Friday or I'll cancel.
"""
triage_ticket(sample_ticket, "alice@example.com")

In [ ]:
sample_ticket = """
Hi, when do parcels usually get delivered?
"""
triage_ticket(sample_ticket, "bob@example.com")